In [20]:
from transformers import pipeline
import torch
import spacy
import gradio as gr


class TextSummarizer:
  def __init__(self):
    self.model= pipeline("summarization", model = "facebook/bart-large-cnn", device = 0 if torch.cuda.is_available() else -1)


  def chunk_text(self, text, max_chunk = 1024):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    current_chunk = []
    total_chunks = []
    for sentence in sentences:
      if len(" ".join(current_chunk + [sentence])) <= max_chunk:
        current_chunk.append(sentence)
      else:
        total_chunks.append(" ".join(current_chunk))
        current_chunk = [sentence]

    if current_chunk:
      total_chunks.append(" ".join(current_chunk))
    return total_chunks


  def summarize(self, text, ratio = 0.2):
    chunks = self.chunk_text(text)
    summarized_text = []
    for chunk in chunks:
      output = self.model(chunk, max_length = int(len(chunk.split())*ratio), min_length = int(len(chunk.split())*ratio*0.5), do_sample = False)
      summarized_text.append(output[0]['summary_text'])
    return " ".join(summarized_text)


summariztxt = TextSummarizer()
interface = gr.Interface(
    fn = summariztxt.summarize,
    inputs = gt.Textbox(lines = 10, placeholder = "Paste you text here..."),
    outputs = "text",
    title = "Text Summarizer"
)

interface = interface.launch(share = True)